In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
rl_kpis = pd.read_csv('rl-kpis.csv')
rl_kpis.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
def knnImputer(data, attribute):
    imputer = KNNImputer()
    if int(data[attribute].isnull().sum()) > 0 and (data[attribute].dtypes == 'int64' or data[attribute].dtypes == 'float64'):
        imputed_data = imputer.fit_transform(pd.DataFrame(data[attribute]))
        data[attribute] = imputed_data
        
    return data

In [4]:
columns = ['severaly_error_second', 'error_second', 'unavail_second', 'avail_time', 'bbe', 'rxlevmax', 'capacity', 'modulation',
           'rlf']

In [5]:
for x in columns:
    rl_kpis = knnImputer(rl_kpis, x)

In [6]:
rl_kpis['freq_band'].fillna('f3', inplace = True)

In [7]:
rl_kpis.drop_duplicates(inplace = True)

In [8]:
rl_kpis.sort_values(['site_id', 'datetime'], ascending = True, inplace = True)

In [9]:
rl_kpis['datetime'] = pd.to_datetime(rl_kpis['datetime'], format = '%Y-%m-%d')

In [10]:
def resampling(data):
    resampled_data = data.resample('1D', on = 'datetime').max()
    resampled_data.dropna(axis = 0, how = 'all', inplace = True)
    resampled_data.drop(columns = 'datetime', inplace = True)
    return resampled_data

In [11]:
resampled_data = pd.DataFrame()

for x, df_site in rl_kpis.groupby('site_id'):
    data = resampling(df_site)
    resampled_data = resampled_data.append(data)

In [12]:
#resampled_data.to_csv('final_rl_data.csv')